In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
import pandas as pd
import time
from tqdm import tqdm
from collections import OrderedDict

In [3]:
path="preprocessed_data/"
train=pd.read_csv(path+"preprocessed_train.csv")
test=pd.read_csv(path+"preprocessed_test.csv")

In [4]:
movies = pd.concat([train,test], ignore_index=True)["title"].unique()

In [5]:
movie_inform = {i:OrderedDict({
  'inform':"",
  'url':0,
  'production_participant':"",
  'summary':'',
  'title_in_naver':'',
  'review':OrderedDict({"id":""})
  }) for i in movies}

error_naver=[]

In [9]:
driver = webdriver.Chrome("/Users/taehoon/chromedriver")
driver.implicitly_wait(time_to_wait=3)

def get_title_link(movie_title):
    search_txt = lambda movie_title:''.join(list(filter(lambda x:x.isdigit()or x.isalpha(),list(movie_title))))
    return f"https://movie.naver.com/movie/search/result.naver?section=movie&query={movie_title}"
    


for _title in tqdm(movie_inform):
    _url = get_title_link(_title)
    driver.get(_url)

    try:
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".etc")))
    except:
        error_naver.append(_title)
        continue

    informs = driver.find_elements_by_class_name("etc")

    for idx, inform in enumerate(informs):
        if "한국" in inform.text:
            movie_inform[_title]['inform']=inform.find_element_by_xpath('..').text
            item = inform.find_element_by_xpath('..').find_element_by_css_selector('dt > a')
            movie_inform[_title]['url']=item.get_attribute('href')
            break
    else:
        movie_inform[_title]['inform']=inform.find_element_by_xpath('..').text
        item = inform.find_element_by_xpath('..').find_element_by_css_selector('dt > a')
        movie_inform[_title]['url']=item.get_attribute('href')

movie_inform, error_naver

SyntaxError: invalid syntax (2007717879.py, line 33)

In [11]:
movie_inform['러브앤서울 복숭아마을 도색소동기']['url']='https://movie.naver.com/movie/bi/mi/basic.naver?code=139612'
movie_inform['대한민국 1%']['url']='https://movie.naver.com/movie/bi/mi/basic.naver?code=73638'


In [13]:
movie_inform['러브앤서울 복숭아마을 도색소동기']['inform']="'대한민국 1% (Miss Staff Sergeant)\n6.29 (참여 1719명)\n드라마, 액션| 한국|105분 |2010\n감독 : 조명남|출연 : 손병호, 임원희, 이아이'"
movie_inform['대한민국 1%']['inform']="'러브&서울 복숭아마을 도색소동기 (AV Idol 2)\n4.58 (참여 12명)\n멜로/로맨스| 일본, 한국|89분 |2014\n감독 : 히데오 조조|출연 : 나나우미 나나, 이토 베니, 황지후, 장대윤'"

In [14]:
movie_inform

{'개들의 전쟁': OrderedDict([('inform',
               '개들의 전쟁\n7.31 (참여 992명)\n액션, 드라마, 코미디| 한국|96분 |2012\n감독 : 조병옥|출연 : 김무열, 진선규'),
              ('url',
               'https://movie.naver.com/movie/bi/mi/basic.naver?code=83960'),
              ('production_participant', ''),
              ('summary', ''),
              ('review', OrderedDict([('id', '')]))]),
 '내부자들': OrderedDict([('inform',
               '내부자들 (Inside Men)\n9.00 (참여 25161명)\n범죄, 드라마| 한국|130분 |2015\n감독 : 우민호|출연 : 이병헌, 조승우, 백윤식'),
              ('url',
               'https://movie.naver.com/movie/bi/mi/basic.naver?code=121788'),
              ('production_participant', ''),
              ('summary', ''),
              ('review', OrderedDict([('id', '')]))]),
 '은밀하게 위대하게': OrderedDict([('inform',
               '은밀하게 위대하게 (Secretly Greatly)\n7.12 (참여 37499명)\n액션, 드라마, 코미디| 한국|123분 |2013\n감독 : 장철수|출연 : 김수현, 박기웅, 이현우, 손현주'),
              ('url',
               'https://movie.naver.com/movie/bi/mi/basic.naver?code=92575')

In [47]:
movie_df = pd.DataFrame(movie_inform).T

In [48]:
movie_df

,inform,url,production_participant,summary,review
개들의 전쟁,"개들의 전쟁\n7.31 (참여 992명)\n액션, 드라마, 코미디| 한국|96분 |...",https://movie.naver.com/movie/bi/mi/basic.nave...,,,{'id': ''}
내부자들,"내부자들 (Inside Men)\n9.00 (참여 25161명)\n범죄, 드라마| ...",https://movie.naver.com/movie/bi/mi/basic.nave...,,,{'id': ''}
은밀하게 위대하게,은밀하게 위대하게 (Secretly Greatly)\n7.12 (참여 37499명)...,https://movie.naver.com/movie/bi/mi/basic.nave...,,,{'id': ''}
나는 공무원이다,나는 공무원이다 (Dangerously Excited)\n6.15 (참여 1540명...,https://movie.naver.com/movie/bi/mi/basic.nave...,,,{'id': ''}
불량남녀,불량남녀\n8.16 (참여 2328명)\n코미디| 한국|108분 |2010\n감독 ...,https://movie.naver.com/movie/bi/mi/basic.nave...,,,{'id': ''}
...,...,...,...,...,...
해에게서 소년에게,해에게서 소년에게 (Under the Sun)\n7.94 (참여 48명)\n드라마|...,https://movie.naver.com/movie/bi/mi/basic.nave...,,,{'id': ''}
울보 권투부,울보 권투부 (A Crybaby Boxing Club)\n8.44 (참여 72명)\...,https://movie.naver.com/movie/bi/mi/basic.nave...,,,{'id': ''}
어떤살인,"어떤살인 (The Lost Choices)\n7.53 (참여 852명)\n범죄, 스...",https://movie.naver.com/movie/bi/mi/basic.nave...,,,{'id': ''}
말하지 못한 비밀,말하지 못한 비밀 (Black Idols)\n4.86 (참여 43명)\n드라마| 한...,https://movie.naver.com/movie/bi/mi/basic.nave...,,,{'id': ''}


In [49]:
movie_df["production_participant"] = movie_df["inform"].apply(lambda x: x.split('\n')[-1])
movie_df["rating"] = movie_df["inform"].apply(lambda x: x.split('\n')[1].split()[0])
movie_df["rating_num"] = movie_df["inform"].apply(lambda x: x.split('\n')[1].split()[2].strip('참여명)'))
movie_df["rating"]

IndexError: list index out of range

In [50]:
movie_df["rating_num"]

KeyError: 'rating_num'